In [ ]:
xxx = '/kaggle/working'
zzz = '/kaggle/working/ComfyUI'

%cd -q $xxx
!curl -sLo $xxx/pantat88.py https://github.com/gutris1/segsmaker/raw/main/kaggle/script/pantat88.py
%run $xxx/pantat88.py
!git clone -q https://github.com/comfyanonymous/ComfyUI
print("【 wait 1/2 】")
%cd -q $zzz
!pip install -q compiletools ffmpeg av Avpy imageio imageio-ffmpeg pypng pyngrok &>/dev/null
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl \
-r requirements.txt &>/dev/null
!pip install -q --force-reinstall torch==2.1.0+cu118 torchvision torchaudio \
--extra-index-url https://download.pytorch.org/whl/cu118 'numpy>=1.16.5,<1.23.0' &>/dev/null

%pull https://github.com/gutris1/segsmaker cui $zzz
%download $zzz/asd/cui.txt
!python $zzz/asd/cui.py > /dev/null

''' Custom Nodes '''
%cd -q $zzz/custom_nodes
%clone $zzz/asd/custom_nodes.txt
%download $zzz/asd/custom_nodes_models.txt
print("【 wait 2/2 】")
%cd -q $zzz
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && \
python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" &>/dev/null

''' ControlNet models '''
!mkdir -p /kaggle/temp/controlnet
%cd -q $zzz/models/controlnet
%download $zzz/asd/controlnet.txt

''' Checkpoint '''
!mkdir -p /kaggle/temp/checkpoints /kaggle/temp/loras
%cd -q $zzz/models/checkpoints
%download https://civitai.com/api/download/models/287641 meichidarkV5.safetensors

''' LoRA '''
%cd -q $zzz/models/loras
%download https://civitai.com/api/download/models/87153  add_detail.safetensors
%download https://civitai.com/api/download/models/171989 detail_slider_v4.safetensors
%download https://civitai.com/api/download/models/62833  Detail_Tweaker.safetensors
%download https://civitai.com/api/download/models/86247  flat2.safetensors

In [ ]:
''' Run ComfyUI '''

ngrok_token = "PASTE_NGROK_TOKEN"

from threading import Timer
from queue import Queue
from pyngrok import ngrok

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)

ngrok_output_queue = Queue()
ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
ngrok_thread.start()
ngrok_thread.join()
print(ngrok_output_queue.get())

%cd -q /kaggle/working/ComfyUI
!python main.py --dont-print-server

In [3]:
''' Delete '''
!rm -rf /kaggle/temp/* /kaggle/working/*